In [ ]:
import anthropic
import base64
import io
from PIL import Image
import numpy as np

# Initialize Anthropic client
client = anthropic.Anthropic(api_key="your-api-key-here")

def frame_to_base64(frame, target_size=(768, 432), quality=85):
    """
    Convert numpy frame to base64 for Claude with resizing
    
    Args:
        frame: numpy array (H, W, C)
        target_size: (width, height) - default 768x432 keeps 16:9 ratio
        quality: JPEG quality 1-100 (higher = better quality, larger size)
    """
    img = Image.fromarray(frame)
    
    # Resize if needed
    if img.size != target_size:
        img = img.resize(target_size, Image.Resampling.LANCZOS)
        print(f"Resized from {frame.shape[:2][::-1]} to {target_size}")
    
    # Use JPEG for better compression on game frames
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG", quality=quality, optimize=True)
    size_kb = len(buffer.getvalue()) / 1024
    print(f"Image size: {size_kb:.1f} KB")
    
    return base64.b64encode(buffer.getvalue()).decode()

def get_claude_action(observation, conversation_history=[], image_config=None):
    """
    Get action from Claude based on observation
    
    Args:
        image_config: dict with 'target_size' and 'quality' keys
    """
    if image_config is None:
        image_config = {'target_size': (768, 432), 'quality': 85}
    
    # print(observation)
    # Create text description
    obs_text = f"""
    BOSS HP: {observation['boss_hp']:.1%}
    PLAYER HP: {observation['player_hp']:.1%}
    DISTANCE: {observation['distance']:.1f}
    BOSS ANIMATION: {observation['boss_animation']}
    PLAYER ANIMATION: {observation['player_animation']}
    
    Available actions:
    0: no-op, 1: forward, 2: backward, 3: left, 4: right, 5: jump
    6: dodge_forward, 7: dodge_backward, 8: dodge_left, 9: dodge_right
    10: interact, 11: attack, 12: use_item
    
    Choose ONE action (just the number). Consider:
    - Attack when close and boss is vulnerable
    - Dodge when boss is attacking
    - Move to maintain good distance
    Respond with just the action number and brief reason.
    """
    
    # Build messages
    messages = conversation_history + [{
        "role": "user",
        "content": [
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": frame_to_base64(
                        observation['frame'], 
                        target_size=image_config['target_size'],
                        quality=image_config['quality']
                    )
                }
            },
            {"type": "text", "text": obs_text}
        ]
    }]
    
    # Get Claude's response
    response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=150,
        messages=messages
    )
    
    response_text = response.content[0].text
    
    # Parse action (extract first number)
    import re
    match = re.search(r'\b([0-9]|1[0-2])\b', response_text)
    action = int(match.group(1)) if match else 0
    
    return action, response_text, messages

# Main game loop
def play_game(env, num_steps=100, keep_history=False, image_config=None):
    """
    Play game with Claude
    
    Args:
        image_config: dict with resolution settings
            Examples:
            - {'target_size': (768, 432), 'quality': 85}  # Balanced (default)
            - {'target_size': (512, 288), 'quality': 75}  # Fast/cheap
            - {'target_size': (1024, 576), 'quality': 90} # High quality
    """
    if image_config is None:
        image_config = {'target_size': (768, 432), 'quality': 85}
    
    print(f"Starting game with image config: {image_config}")
    
    obs, info = env.reset()
    conversation_history = []
    
    for step in range(num_steps):
        print(f"\n{'='*60}")
        print(f"STEP {step}")
        print(f"{'='*60}")
        
        # Get action from Claude
        history = conversation_history if keep_history else []
        action, reasoning, messages = get_claude_action(obs, history, image_config)
        
        # Debug output
        print(f"Boss HP: {obs['boss_hp']:.1%} | Player HP: {obs['player_hp']:.1%}")
        print(f"Distance: {obs['distance']:.1f}")
        print(f"\nClaude's reasoning:\n{reasoning}")
        print(f"\nChosen action: {action}")
        
        # Update conversation history if keeping context
        if keep_history:
            conversation_history = messages + [{
                "role": "assistant",
                "content": reasoning
            }]
            # Keep last 10 exchanges to avoid token limits
            if len(conversation_history) > 20:
                conversation_history = conversation_history[-20:]
        
        # Step environment
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        print(f"Reward: {reward:.2f}")
        
        if done:
            print("\n🎮 Episode finished!")
            break
    
    return obs

# Usage examples:

# Default - balanced quality/speed
# play_game(env, num_steps=50)

# Fast mode - smaller images, faster API calls, cheaper
# play_game(env, num_steps=50, image_config={'target_size': (512, 288), 'quality': 75})

# High quality mode - better for visual details
# play_game(env, num_steps=50, image_config={'target_size': (1024, 576), 'quality': 90})

# Test single frame size
# obs = env.reset()
# print(f"Original frame shape: {obs['frame'].shape}")
# _ = frame_to_base64(obs['frame'], target_size=(768, 432), quality=85)

In [17]:
play_game(env, num_steps=50)

🎮 Starting game with image config: {'target_size': (768, 432), 'quality': 85}

STEP 0
  📐 Resized from (3840, 2160) to (768, 432)
  💾 Image size: 58.4 KB
Boss HP: 100.0% | Player HP: 100.0%
Distance: 70.9

Claude's reasoning:
1

**Reason:** Distance is 70.9 units, which is too far to engage. Boss animation 2002000 appears to be an idle/neutral state. Need to close the gap to get into combat range. Moving forward to approach the boss.

Chosen action: 1
Reward: 0.00

STEP 1
  📐 Resized from (3840, 2160) to (768, 432)
  💾 Image size: 61.7 KB
Boss HP: 100.0% | Player HP: 100.0%
Distance: 42.7

Claude's reasoning:
1

**Reason:** Distance is 42.7 units, which is too far for combat. Boss animation 2003029 suggests it's in a neutral state. Need to close the gap to engage Margit effectively. Moving forward to get into attack range.

Chosen action: 1
Reward: -198.00

STEP 2
  📐 Resized from (3840, 2160) to (768, 432)
  💾 Image size: 59.0 KB
Boss HP: 100.0% | Player HP: 91.1%
Distance: 35.9

Clau

{'frame': array([[[11, 22, 10],
         [11, 22, 10],
         [11, 22, 10],
         ...,
         [22, 28,  8],
         [20, 25,  8],
         [18, 22,  7]],
 
        [[11, 22, 10],
         [11, 22, 10],
         [11, 22, 10],
         ...,
         [21, 28,  9],
         [21, 26,  8],
         [19, 24,  8]],
 
        [[11, 22, 10],
         [11, 22, 10],
         [11, 23, 10],
         ...,
         [22, 30, 12],
         [21, 29, 11],
         [21, 29,  9]],
 
        ...,
 
        [[ 4,  6,  1],
         [ 3,  6,  1],
         [ 3,  5,  1],
         ...,
         [23, 32, 16],
         [20, 34, 17],
         [24, 37, 23]],
 
        [[ 4,  6,  1],
         [ 4,  6,  1],
         [ 3,  5,  1],
         ...,
         [30, 31,  9],
         [22, 28, 18],
         [24, 28, 22]],
 
        [[ 3,  6,  1],
         [ 3,  4,  1],
         [ 3,  4,  1],
         ...,
         [33, 38, 40],
         [28, 34, 19],
         [26, 43, 21]]], shape=(2160, 3840, 3), dtype=uint8),
 'boss_hp'